In [1]:
from rdflib import Namespace, Graph
from buildingmotif import BuildingMOTIF
from buildingmotif.dataclasses import Model, Library
from buildingmotif.database.tables import Base as BuildingMotif_tables_base

This notebook walks through how to write templates and evaluate them to create building models

## Writing Templates

A template is a function that generates a graph. The template definition provides the structure of the graph and allows the content of the graph to be determined in part through the use of *parameters*. Parameters have a name (typically short and descriptive) and can be required or optional. Parameters are identified by their prefix: `urn:___param___#`

Templates are most easily written as YAML documents. A YAML file can contain more than one template. The names of the templates are at the top level of the YAML file as "keys".  The associated values are the content of the template. Templates can have 3 fields:
- `body`: a turtle-serialized graph defining the structure of what will be generated
- `optional`: a list of parameters which are optional in the template evaluation. If not bound, they will be removed from the generated graph
- `dependencies`: a list of dependencies on other templates

To start, let's look at a simple example without optional params or dependencies:

```yaml
# tutorial/templates.yml
my-thermostat:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasLocation P:room .
```

This template has two parameters: `name` and `room`. **All templates are required to have a `name` parameter.** This acts as the "root" of the template.

The `my-thermostat` template describes the thermostat as having a location, but no further properties. We also don't know what kind of thing `P:room` should be bound to when the template is used. Let's address the second problem first by introducing a dependency

```yaml
# tutorial/templates.yml
my-thermostat:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasLocation P:room .
  dependencies:
  - template: https://brickschema.org/schema/Brick#Room
    library: https://brickschema.org/schema/1.3/Brick
    args: {"name": "room"}
```

The `dependencies` key contains a list of documents which describe the dependencies of this template on other templates. The single entry above states that the `my-thermostat` template is dependent upon the `https://brickschema.org/schema/Brick#Room` template (automatically produced by importing the Brick ontology as a library), and the `name` parameter of the `Room` template is bound to the value of the `room` parameter in this template.

Let's now add another template to our library, which `my-thermostat` will also depend on, which defines some of the points we want to see on our thermostat:

```yaml
my-tstat-points:
  body: >
     @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasPoint P:temp, P:sp, P:co2 .
  dependencies:
    - template: https://brickschema.org/schema/Brick#Temperature_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "temp"}
    - template: https://brickschema.org/schema/Brick#Temperature_Setpoint
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "sp"}
    - template: https://brickschema.org/schema/Brick#CO2_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "co2"}
```

(There is syntax sugar for the above pattern)

We can now have our original `my-thermostat` template *depend* on this template. By binding this template to the `name` paramter of our original template, we are essentially composing the two templates together.

The full template library is now as follows:

```yaml
# tutorial/templates.yml
my-thermostat:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasLocation P:room .
  dependencies:
  - template: https://brickschema.org/schema/Brick#Room
    library: https://brickschema.org/schema/1.3/Brick
    args: {"name": "room"}
  - template: my-tstat-points
    args: {"name": "name"}
    
my-tstat-points:
  body: >
    @prefix P: <urn:___param___#> .
    @prefix brick: <https://brickschema.org/schema/Brick#> .
    P:name a brick:Thermostat ;
        brick:hasPoint P:temp, P:sp, P:co2 .
  dependencies:
    - template: https://brickschema.org/schema/Brick#Temperature_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "temp"}
    - template: https://brickschema.org/schema/Brick#Temperature_Setpoint
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "sp"}
    - template: https://brickschema.org/schema/Brick#CO2_Sensor
      library: https://brickschema.org/schema/1.3/Brick
      args: {"name": "co2"}
```

In [2]:
# load the template library as follows:
# 1. create the BuildingMOTIF instance
bm = BuildingMOTIF("sqlite://") # in-memory
BuildingMotif_tables_base.metadata.create_all(bm.engine)
# 2a. load dependencies (otherwise buildingMOTIF will complain)
_ = Library.load(ontology_graph="../libraries/brick/Brick-subset.ttl")
# 2b. load the library from the directory
lib = Library.load(directory="./tutorial")
# the name of the library will be name of the directory
print(f"Loaded library named {lib.name}")

Loaded library named tutorial


In [3]:
# now we can traverse the library to see what's there
for template in lib.get_templates():
    print(f"Template '{template.name}' has parameters {template.parameters}")
    print("The body of the template looks like this:")
    bindings, graph = template.fill(Namespace("urn:example/"))
    print(graph.serialize(format='turtle'))

Template 'my-thermostat' has parameters {'name', 'room'}
The body of the template looks like this:
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example/name_df73c18f> a brick:Thermostat ;
    brick:hasLocation <urn:example/room_3236f7b1> .


Template 'my-tstat-points' has parameters {'temp', 'co2', 'name', 'sp'}
The body of the template looks like this:
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example/name_54243d8f> a brick:Thermostat ;
    brick:hasPoint <urn:example/co2_698e8496>,
        <urn:example/sp_82db8616>,
        <urn:example/temp_c4583cac> .




## Evaluating Templates

Generating a graph from a template is called *evaluating* the template. Template evaluation requires a *binding* of template parameters to values. Values can be either RDF URIs or Literals.

To start, grab a template from the library. This can be done by name:

In [4]:
templ = lib.get_template_by_name("my-thermostat")
print(templ.name, templ.parameters)

my-thermostat {'name', 'room'}


The `.fill` method will invent names for each of the parameters in the template so you can see what it looks like as a graph:

In [5]:
bindings, graph = templ.fill(Namespace("urn:example/"))
print(f"BuildingMOTIF invented bindings: {bindings}")
print("The resulting graph looks like:")
print(graph.serialize(format='turtle'))

BuildingMOTIF invented bindings: {'name': rdflib.term.URIRef('urn:example/name_8382dc94'), 'room': rdflib.term.URIRef('urn:example/room_7e09fdae')}
The resulting graph looks like:
@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:example/name_8382dc94> a brick:Thermostat ;
    brick:hasLocation <urn:example/room_7e09fdae> .




You'll notice that the dependencies do not appear in this graph. To include these, use `.inline_dependencies` to get a new template:

In [6]:
inlined_tstat = templ.inline_dependencies()
print("need bindings for:", inlined_tstat.parameters)

need bindings for: {'sp', 'temp', 'co2', 'name', 'room'}


Usually, we want to use our own names when evaluating a template. To do this, we use the `.evaluate` method. `.evaluate()` takes a dictionary of parameter names to parameter values as an argument.

We know from above that the parameters for the template are 'name', 'temp', 'room', 'sp', and 'co2'. Let's invent some entities for those. We will create a new namespace to hold those entities as well.

In [7]:
# new namespace
BLDG = Namespace("urn:my-building/")

bindings = {
    'name': BLDG['tstat1'],
    'temp': BLDG['temp_sensor1'],
    'room': BLDG['room_410'],
    'sp': BLDG['temp_setpoint1'],
    'co2': BLDG['co2_sensor'],
}

graph = inlined_tstat.evaluate(bindings)

If we did not provide all of the required parameters, we would get another Template back instead of a graph. Here, we *have* provided all of the required bindings so we get the graph

In [8]:
assert isinstance(graph, Graph)

In [9]:
# we can now look at the resulting graph
print(graph.serialize(format='turtle'))

@prefix brick: <https://brickschema.org/schema/Brick#> .

<urn:my-building/tstat1> a brick:Thermostat ;
    brick:hasLocation <urn:my-building/room_410> ;
    brick:hasPoint <urn:my-building/co2_sensor>,
        <urn:my-building/temp_sensor1>,
        <urn:my-building/temp_setpoint1> .

<urn:my-building/co2_sensor> a brick:CO2_Sensor .

<urn:my-building/room_410> a brick:Room .

<urn:my-building/temp_sensor1> a brick:Temperature_Sensor .

<urn:my-building/temp_setpoint1> a brick:Temperature_Setpoint .




## Assembling a Model

We first create a model in BuildingMOTIF which will represent our building. All models need a name. It is helpful and idiomatic (but not necessary) to name the model with the namespace that will contain the entities:

In [10]:
bldg = Model.create(BLDG)
print(bldg.graph.serialize()) # basic graph metadata!

@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:my-building/> a owl:Ontology .




Use the `add_graph` method to append additional metadata to the graph

In [11]:
bldg.add_graph(graph) # 'graph' is the vav we created above. This is largely idempotent
print(bldg.graph.serialize()) # now contains our tstat

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:my-building/> a owl:Ontology .

<urn:my-building/tstat1> a brick:Thermostat ;
    brick:hasLocation <urn:my-building/room_410> ;
    brick:hasPoint <urn:my-building/co2_sensor>,
        <urn:my-building/temp_sensor1>,
        <urn:my-building/temp_setpoint1> .

<urn:my-building/co2_sensor> a brick:CO2_Sensor .

<urn:my-building/room_410> a brick:Room .

<urn:my-building/temp_sensor1> a brick:Temperature_Sensor .

<urn:my-building/temp_setpoint1> a brick:Temperature_Setpoint .




Templates allow the user to populate models using tabular input rather than by explicit graph construction. Consider the following CSV file:


| name | room | co2 | temp | sp |
|------|------|-----|------|----|
|tstat2|room345|co2-345|temp-345|sp-345|
|tstat3|room567|cow-567|temp-567|sp-567|

In a few lines of Python, we can read this CSV file and use its columns to instantiate more thermostats

In [12]:
import csv
with open("tutorial/data.csv") as f:
    rdr = csv.DictReader(f)
    for line in rdr:
        graph = inlined_tstat.evaluate({
            k: BLDG[v] for k,v in line.items()
        })
        bldg.add_graph(graph)

In [13]:
print(bldg.graph.serialize()) # now contains our tstats

@prefix brick: <https://brickschema.org/schema/Brick#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .

<urn:my-building/> a owl:Ontology .

<urn:my-building/tstat1> a brick:Thermostat ;
    brick:hasLocation <urn:my-building/room_410> ;
    brick:hasPoint <urn:my-building/co2_sensor>,
        <urn:my-building/temp_sensor1>,
        <urn:my-building/temp_setpoint1> .

<urn:my-building/tstat2> a brick:Thermostat ;
    brick:hasLocation <urn:my-building/room345> ;
    brick:hasPoint <urn:my-building/co2-345>,
        <urn:my-building/sp-345>,
        <urn:my-building/temp-345> .

<urn:my-building/tstat3> a brick:Thermostat ;
    brick:hasLocation <urn:my-building/room567> ;
    brick:hasPoint <urn:my-building/cow-567>,
        <urn:my-building/sp-567>,
        <urn:my-building/temp-567> .

<urn:my-building/co2-345> a brick:CO2_Sensor .

<urn:my-building/co2_sensor> a brick:CO2_Sensor .

<urn:my-building/cow-567> a brick:CO2_Sensor .

<urn:my-building/room345> a brick:Room .

<urn:my-bu

Validate the model against the shapes in the `tutorial` library:

In [14]:
bldg.validate([lib.get_shape_collection()])

False